# **Question 01**

## **Problem Statement**
You are given a **4x4 sliding tile puzzle** consisting of **15 numbered tiles (1-15)** and **one empty space (0)**.  
Your task is to implement the **A\* search algorithm** to find the **shortest sequence of moves** that transforms the given **start state** into the **goal state** using the **Manhattan distance heuristic**.

---

## **Initial and Goal States**
### **Start State (Given)**
```python
start_state = [
    [1, 2, 3, 4],
    [5, 6, 0, 8],
    [9, 10, 7, 11],
    [13, 14, 15, 12]
]

goal_state = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]
```
### **Expected Output:**

*     Sequence of moves required to reach the goal (e.g., "Move tile 7 left").
*     Final grid configuration after applying the moves.

### **Constraints:**
1.    You can only move the empty space (0) to adjacent positions (up, down, left, right).
2.    The goal is to minimize the number of moves to reach the solved state.
3.    **Heuristic Function:** The Manhattan distance between each tile’s current position and its goal position. The Manhattan distance for each tile is the sum of the horizontal and vertical distance between its current and goal positions.


In [28]:
def manhattan_distance(state):
    """
    Compute the Manhattan distance heuristic for a given puzzle state.

    - Each tile's distance to its goal position is calculated as:
      h(n) = |current_row - goal_row| + |current_col - goal_col|

    - Sum up distances for all tiles (ignoring the empty space).

    Return:
        int: Total heuristic cost (h(n))
    """
    distance = 0
    for i in range(len(state)):
        for j in range(len(state[i])):
            if state[i][j] != 0:  # Ignore the empty space
                # Find the goal position of the current tile
                goal_row = (state[i][j] - 1) // len(state)
                goal_col = (state[i][j] - 1) % len(state[i])
                # Calculate the Manhattan distance
                distance += abs(i - goal_row) + abs(j - goal_col)
    return distance

In [ ]:
import heapq

def get_neighbors(state):
    """
    Generate all valid neighboring states by moving the empty space (0).
    """
    neighbors = []
    rows, cols = len(state), len(state[0])
    directions = [(0, 1, "right"), (1, 0, "down"), (0, -1, "left"), (-1, 0, "up")]  # (dr, dc, move)

    
    empty_row, empty_col = -1, -1
    for i in range(rows):
        for j in range(cols):
            if state[i][j] == 0:
                empty_row, empty_col = i, j
                break

    
    for dr, dc, move in directions:
        new_row, new_col = empty_row + dr, empty_col + dc
        if 0 <= new_row < rows and 0 <= new_col < cols:
            
            new_state = [row[:] for row in state]  
            new_state[empty_row][empty_col], new_state[new_row][new_col] = (
                new_state[new_row][new_col],
                new_state[empty_row][empty_col],
            )
            
            tile_moved = state[new_row][new_col]
            move_description = f"Move tile {tile_moved} {move}"
            neighbors.append((new_state, move_description))
    return neighbors

def a_star_search(start, goal):
    """
    ### TODO:
    Implement A* search algorithm to find the shortest path from start state to goal state.

    - Use a priority queue (min-heap) to explore states.
    - Select the state with the lowest f(n) = g(n) + h(n).
    - Generate all valid moves by shifting the empty space (0).

    Return:
        list: Sequence of moves to reach the goal.
    """

    # Priority queue: (f(n), g(n), state, path, moves)
    heap = [(manhattan_distance(start), 0, start, [], [])]
    heapq.heapify(heap)

    
    visited = set()

    while heap:
        f_n, g_n, current_state, path, moves = heapq.heappop(heap)

        if current_state == goal:
            return moves, current_state  

        if tuple(map(tuple, current_state)) in visited:
            continue  

        visited.add(tuple(map(tuple, current_state)))

        # Generate neighboring states
        for neighbor, move_description in get_neighbors(current_state):
            if tuple(map(tuple, neighbor)) not in visited:
                
                h_n = manhattan_distance(neighbor)
                heapq.heappush(
                    heap,
                    (g_n + 1 + h_n, g_n + 1, neighbor, path + [neighbor], moves + [move_description]),
                )

    return None, None  



In [30]:


start_state = [
    [1, 2, 3, 4],
    [5, 6, 0, 8],
    [9, 10, 7, 11],
    [13, 14, 15, 12]
]

goal_state = [
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 0]
]

# solution_moves = a_star_search(start_state, goal_state)
# #print("Solution Moves:", solution_moves)

moves, final_state = a_star_search(start_state, goal_state)
if moves:
    print("Solution found! Sequence of moves:")
    for move in moves:
        print(move)
    print("\nFinal grid configuration:")
    for row in final_state:
        print(row)
else:
    print("No solution found.")


Solution found! Sequence of moves:
Move tile 7 down
Move tile 11 right
Move tile 12 down

Final grid configuration:
[1, 2, 3, 4]
[5, 6, 7, 8]
[9, 10, 11, 12]
[13, 14, 15, 0]


# **Question 02**

This is a classic optimization problem, where we want to maximize the overall rating of the product while keeping the cost under a certain budget. We have a large dataset of customer reviews and ratings for a product, and we want to identify the most important features of the product based on customer feedback. Our goal is to maximize the overall rating of the product by improving the most important features. We want to find the best sequence of feature improvements that maximize the overall rating while keeping the cost under a certain
budget.

In [9]:
import random

# feature improvements and their corresponding costs and ratings
features = {
    "feature1": {"cost": 100, "rating": 3.5},
    "feature2": {"cost": 200, "rating": 4.2},
    "feature3": {"cost": 150, "rating": 4.0},
    "feature4": {"cost": 300, "rating": 3.8},
    "feature5": {"cost": 250, "rating": 4.5},
    "feature6": {"cost": 350, "rating": 3.6}
}

# budget for feature improvements
budget = 200

In [10]:

def evaluate(state):
    """
    - Calculate the total cost of the selected features.
    - If the total cost is greater than the budget, return 0 (since we cannot exceed the budget).
    - Otherwise, calculate and return the total rating improvement from the selected features.
    """
    total_cost = sum(features[feature]["cost"] for feature in state)
    if total_cost > budget:
        return 0
    total_rating = sum(features[feature]["rating"] for feature in state)
    return total_rating


def find_best_neighbor(state, remaining_budget):
    best_neighbor = state
    best_score = evaluate(state)

    for feature_name in features:
        if feature_name not in state and features[feature_name]["cost"] <= remaining_budget:
            neighbor = state + [feature_name]
            score = evaluate(neighbor)

            if score > best_score:
                best_score = score
                best_neighbor = neighbor

    return best_neighbor

Now, apply the hill climbing algorithm using the following steps provided


1.   start with a random initial state

2.   iterate until a local maximum is reached

3. evaluate the neighboring states and select the one that improves the evaluation function the most

4. update the current state with the selected neighboring state
5. update the best state if the current state is better



In [11]:
def Hill_Climb():
    selected_features = []
    remaining_budget = budget

    while True:
       
        new_state = find_best_neighbor(selected_features, remaining_budget)

      
        if new_state == selected_features:
            break

        
        selected_features = new_state
        remaining_budget = budget - sum(features[feature]["cost"] for feature in selected_features)

    return selected_features




In [14]:

best_state = Hill_Climb()

print("Best state:", best_state)
print("Total rating:", evaluate(best_state))
print("Total cost:", sum(features[feature]["cost"] for feature in best_state))

Best state: ['feature2']
Total rating: 4.2
Total cost: 200
